In [ ]:
import dotenv
from rest import OdooAPIKey, OdooAPIBase
import json
env_path = "conf/.env"
dotenv.load_dotenv(env_path)
key = OdooAPIKey.prod()
client = OdooAPIBase(key)

def to_json(data):
    return json.dumps(data, indent=4)



In [ ]:
# 获取所有客户
domain = [
    ('customer_rank', '>=', 1)
]
data = client.client.search_read('res.partner', [domain], {})
print(len(data))
print(to_json(data[10]))

In [ ]:
data = client.client.search_read('res.partner', [domain], {})
print(len(data))
print(data[10])

In [ ]:
# 获取所有价格表
domain = [('active', '=', True)]
data = client.client.search_read('product.pricelist', [domain], {"fields": ['id', 'name', 'active']})
print(f"Total number of pricelists: {len(data)}")
# print(to_json(data))
list_pricelists = data 

In [ ]:
def get_pricelist_item_data(pricelist_id):
    domain = [('pricelist_id', '=', pricelist_id)]
    pricelist_item_data = client.client.search_read('product.pricelist.item', [domain], {"fields": ["id", "company_id", 
                                                                                     "fixed_price", "name", "currency_id", 
                                                                                     "min_quantity", "product_tmpl_id", "product_id"]})
    return pricelist_item_data

pricelist_id = data[3]['id']
pricelist_item_data = get_pricelist_item_data(pricelist_id)
print(len(pricelist_item_data))
print(to_json(pricelist_item_data))

# VIP-Odoo价格表同步
把VIP上的价格表上传到Odoo

In [ ]:
import pandas as pd
import json 

# 获取VIP上的价格表
with open('temp/vip_price.json', 'r', encoding='utf-8') as f:
    data = json.load(f)['data']

# Data Preprocessing to prepare for Odoo API
df = pd.DataFrame().from_dict(data)
pattern = r"^(.*?)(PK\d+)$"
# 从article_number中提取reference和size 
df['internal_reference'] = df['article_number'].str.extract(pattern, expand=False)[0]
df['sales_units'] = df['article_number'].str.extract(pattern, expand=False)[1].str.replace('PK', '')
# if internal_reference is NaN, set it to article_number
df['internal_reference'] = df['internal_reference'].fillna(df['article_number'])
# if sales_units is NaN, set it to 1
df['sales_units'] = df['sales_units'].fillna(1)
# convert sales_units to int
df['min_quantity'] = df['sales_units'].astype(int)
df['custom_price'] = df['custom_price'] / df['min_quantity']
df['std_price_a'] = df['std_price_a'] / df['min_quantity']
df['std_price_b'] = df['std_price_b'] / df['min_quantity']
del df['article_number']
df['key'] = df['group_name'] + '_' + df['internal_reference'].astype(str)
df.sort_values(by='group_name', inplace=True)
df.reset_index(drop=True, inplace=True)
list_group_names = df['group_name'].unique().tolist()
df_vip_pricelist_items = df.copy()
df_vip_pricelist_items.sample(20)


In [ ]:
print(f"Total number of groups in VIP: {len(list_group_names)}")
for i, name in enumerate(list_group_names):
    print(f"{i+1}. {name}")

In [ ]:
# 批量获取Odoo上的价格表
vip_pricelist_names = [name.strip() for name in list_group_names]
print(f"Total VIP pricelist names: {len(vip_pricelist_names)}")

domain = [('name', 'in', vip_pricelist_names)]
pricelist_ids = client.client.search('product.pricelist', [domain], {})
print(f"Total Odoo pricelist ids: {len(pricelist_ids)}")

pricelist_details = client.client.read('product.pricelist', [pricelist_ids], {'fields': ['id', 'name', 'active']})
odoo_pricelist_names = [pricelist_detail['name'] for pricelist_detail in pricelist_details]

print("Odoo pricelist names:")
for i, name in enumerate(odoo_pricelist_names):
    print(f"{i+1}. {name}")

map_pricelist_name_to_object = {pricelist_detail['name']: pricelist_detail for pricelist_detail in pricelist_details}


In [ ]:
# 分类：依据是vip价格表是否在odoo上
vip_pricelist_names_not_in_odoo = [name for name in vip_pricelist_names if name not in odoo_pricelist_names]
print(f"Total VIP pricelist names not in Odoo: {len(vip_pricelist_names_not_in_odoo)}")
for i, name in enumerate(vip_pricelist_names_not_in_odoo):
    print(f"\t{i+1}. {name}")
print("-"*100)
vip_pricelist_names_in_odoo = [name for name in vip_pricelist_names if name in odoo_pricelist_names]
print(f"Total VIP pricelist names in Odoo: {len(vip_pricelist_names_in_odoo)}")
for i, name in enumerate(vip_pricelist_names_in_odoo):
    print(f"\t{i+1}. {name}")


In [ ]:
# 获取所有price_list的items
fields = ["id", "company_id", "pricelist_id",
            "fixed_price", "name", "currency_id", 
            "min_quantity", "product_tmpl_id", "product_id"]
domain = [('name', 'in', vip_pricelist_names_in_odoo)]
pricelist_item_data = client.client.search_read('product.pricelist.item', [domain], {"fields": fields})
print(len(pricelist_item_data))
print(pricelist_item_data)

In [ ]:
import re 
import pandas as pd
from pydantic import BaseModel
from typing import Union

# 数据验证和标准化
# 定义pricelist_item的数据结构
class PricelistItem(BaseModel):
    id: int
    pricelist_id: int
    pricelist_name: str
    company_id: int
    company_name: str
    fixed_price: float
    name: str
    currency: str
    min_quantity: int
    product_tmpl_id: int
    product_tmpl_name: str
    default_code: str    
    product_id: Union[int, None]
    product_name: Union[str, None]

pricelist_items = []
for item in pricelist_item_data:
    group = re.search(r'\[(.*?)\]', item['product_tmpl_id'][1])
    if group:
        default_code = group.group(1)
    else:
        default_code = ""    
    pricelist_items.append(PricelistItem(
        id=item['id'],
        pricelist_id=item['pricelist_id'][0],
        pricelist_name=item['pricelist_id'][1].replace('(EUR)', '').strip(),
        company_id=item['company_id'][0],
        company_name=item['company_id'][1],
        fixed_price=item['fixed_price'],
        name=item['name'],
        currency=item['currency_id'][1],
        min_quantity=item['min_quantity'],
        product_tmpl_id=item['product_tmpl_id'][0],
        product_tmpl_name=item['product_tmpl_id'][1],
        default_code=default_code,
        product_id= item['product_id'][0] if item['product_id'] else None,
        product_name=item['product_id'][1] if item['product_id'] else None
    ))

print(pricelist_items[10])
df_odoo_pricelist_items = pd.DataFrame.from_dict([item.model_dump() for item in pricelist_items])
df_odoo_pricelist_items['key'] = df_odoo_pricelist_items['pricelist_name'] + '_' + df_odoo_pricelist_items['default_code']
df_odoo_pricelist_items[['pricelist_name', 'default_code', 'fixed_price', 'min_quantity', 'product_tmpl_name', 'currency',  'key']].sample(20)

In [23]:
# 利用表连接, 对比pricelist_item和vip_price，找出需要创建或更新的items
df_merged = df_vip_pricelist_items.merge(df_odoo_pricelist_items, on='key', how='left')
df_merged_filtered = df_merged[df_merged['group_name'].isin(vip_pricelist_names_in_odoo)].copy()
df_merged_filtered['action'] = df_merged_filtered['pricelist_id'].isna()
df_merged_filtered['action'] = df_merged_filtered['action'].map({True: 'create', False: 'update'})
df_merged_filtered.to_excel('temp/vip_odoo_pricelist_items.xlsx', index=False)

In [ ]:
# 获取所有的odoo product templates, 并建立映射
domain = [('active', '=', True)]
product_templates = client.client.search_read('product.template', [domain], {'fields': ['id', 'name', 'default_code']})
print(f"Total number of product templates: {len(product_templates)}")
print(product_templates)
map_product_template_code_to_object = {pt['default_code']: pt for pt in product_templates}
print(map_product_template_code_to_object)

In [ ]:
import time 
# 更新价格表items
debug = True
df_pricelist_items_update = df_merged_filtered[df_merged_filtered['action'] == 'update'].copy()
print(f"Total number of pricelist items to update: {len(df_pricelist_items_update)}")
# df_pricelist_items_update = df_pricelist_items_update[df_pricelist_items_update['pricelist_id'] == 23]
for i, row in enumerate(df_pricelist_items_update.itertuples()):
    pricelist_item_id = int(row.id)
    update_vals = {        
        'fixed_price': row.custom_price,
        'min_quantity': row.min_quantity_x,
        'compute_price': 'fixed',
    }
    print(f"{i+1}. Updating Pricelist: {row.group_name}")
    print(f" ** Pricelist Item ID: {pricelist_item_id}")
    print(f" ** Update Values: {update_vals}")
    if not debug:
        client.client.write('product.pricelist.item', [[pricelist_item_id], update_vals])
        time.sleep(0.2)


In [ ]:
# 创建价格表items

df_pricelist_items_create = df_merged_filtered[df_merged_filtered['action'] == 'create'].copy()
print(f"Total number of pricelist items to create: {len(df_pricelist_items_create)}")
products_not_found = []
# df_pricelist_items_create = df_pricelist_items_create[df_pricelist_items_create['group_name'] == 'Gesundheits- und Pflegezentrum Goldenherz']
for i, row in enumerate(df_pricelist_items_create.itertuples()):
    try:       
        product_template_obj = map_product_template_code_to_object[row.internal_reference]    
    except KeyError:
        print(f"Product template not found for {row.internal_reference}")
        products_not_found.append(row.internal_reference)
        continue

    pricelist_obj = map_pricelist_name_to_object[row.group_name]
    new_item_vals = {
        'pricelist_id': pricelist_obj['id'],   # Zahnarztpraxis am Siemersplatz
        'product_tmpl_id': product_template_obj['id'],   # Deppe, product_tmpl     
        'fixed_price': row.custom_price,
        'min_quantity': row.min_quantity_x,     
        'compute_price': 'fixed',    # 固定价格
    }

    print(f"{i+1}. Creating item in Pricelist: {row.group_name}")
    print(f" ** Product: [{product_template_obj['default_code']}] {product_template_obj['name']}")
    print(f" ** New Item Values: {new_item_vals}")
    if not debug:
        client.client.create('product.pricelist.item', [new_item_vals])
        time.sleep(0.2)



In [ ]:
print(f"Total number of products not found: {len(products_not_found)}")
print(products_not_found)
